# Grade: /100 pts

# Assignment 06: Feature Selection and Regularization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', 500)

%matplotlib inline

# You're Still a Data Scientist!

Your models from the last assignment really impressed some in the management in your football club. Now that you have learned the art of regularization, your boss thinks you should do equally well with much less data. This will save a lot of money the next time around. This time you only get a data set with 3000 observations.

## Question 1: Preprocessing (5 pts)
Tasks:
* Load the data present in 'footballer_small.csv' using the pandas library and store the loaded data in a dataframe
* Drop the variables: 'ID','club','club_logo','flag', 'nationality','photo','potential', 'birth_date'
* Dummy code the variables: work_rate_att, work_rate_def, preferred_foot. **Because we are running a regularized model, we do not want to drop the first column**. If you want to understand why this is, have a look at the Jupyter notebook (`Ridge_And_Dummycoding.ipynb`).  
* Get a test data set of size 500 - to make results comparable to solutions, set random_state = 0
* visualize all variables of the first 50 observations of the Training data set as an image (see Lab06_followalong). You can also look at it as a data frame. How are the different variables scaled? Which variables have high and which ones have low values?    

To make sure that you get a good start - check the solutions from Assignment 4. And make sure you can apply these steps flexibly and quickly. **You will need it for the midterm!**

In [ ]:
df = pd.read_csv('footballer_small.csv')

# Drop the aformentioned columns
model_data = df.drop(['ID','club','club_logo','flag', 'nationality','photo','potential', 'birth_date'], axis = 'columns')

# In order to get dummies, convert categorical data to categorical type
model_data['work_rate_att'] = pd.Categorical(model_data.work_rate_att, categories=['Low','Medium','High'])
model_data['work_rate_def'] = pd.Categorical(model_data.work_rate_def, categories=['Low','Medium','High'])
model_data['preferred_foot'] = pd.Categorical(model_data.preferred_foot, categories = ['Left','Right'])

# Dummies, dropping the first category
model_data = pd.get_dummies(model_data, drop_first=False)

y = model_data.overall
X = model_data.drop('overall', axis = 'columns')

# Random state assures that folds are consistent across models
Xtrain, Xtest, ytrain, ytest = train_test_split(X,
                                                y, 
                                                test_size = 500, 
                                                random_state = 0)
print('Training set size:',Xtrain.shape)

plt.imshow(Xtrain[:50])
df.head()


**Your written answer here**
0-9 and 12-39 have high values , the rest low

## Question 2: Standardization  (10 pts)
When using regularized regression models, the scaling of the different regressors can influence the results dramatically (see lectures). One simple solution is to standardize all features before estimating the model, so that no feature can dominate others due to differences in feature scales. 

a) Use the sklearn class `StandardScaler` to produce a z-scale version of your training data set. Again visualize the the first 50 observations an image. Compare to the plot that you got in Question 1. What do you observe? 

b) Plot a histogram of the second column (height_cm) of the non-standarized and standardized training set. What is the mean and variance of the standardized training set. 

c) Build a model `pipeline` that first standarizes all the features in the training set and then fits a `LinearRegression` model. 


In [ ]:
scaler = StandardScaler()

transformed_X = scaler.fit_transform(Xtrain)
plt.imshow(transformed_X[:50,])

**Written answer here** Greater variation within variables

In [ ]:
# b) Plot historgram 
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
ax = sns.distplot(Xtrain.stamina,
                 bins=50,
                 kde=True,
                 color='skyblue',
                 hist_kws={"linewidth": 15,'alpha':1})
ax.set(xlabel='stamina', ylabel='frequency', title="Before Standardization")

**Written answer here**
After standardization, the data is more centered around 0 representing the mean and concentrated within one std from the mean being the variance.

In [ ]:
# c: Build pipeline
model_pipeline = Pipeline([
    ('standardize', StandardScaler()),
    ('reg', sk.linear_model.LinearRegression())
])
standardizer_step = model_pipeline.named_steps['standardize']
transformed_X = standardizer_step.fit_transform(Xtrain)
plt.subplot(1, 2, 2)
ax = sns.distplot(transformed_X[:, Xtrain.columns.get_loc("stamina")],
                 bins=50,
                 kde=True,
                 color='skyblue',
                 hist_kws={"linewidth": 15,'alpha':1})
ax.set(xlabel='stamina', ylabel='frequency', title="After Standardization")

## Question 3: Comparing different complex features (10 pts)
In this task, we will first consider a model that includes all the variables in the data AND all quadratic terms (i.e. each features to the power of two, and the products (interactions) between all possible pairs of features. 

a) Generate a design matrix for the model. You can use sklearn's `PolynomialFeatures` to do the job. Because sklearn's linear models have the option to fit the intercept, internally, set the `include_bias` option to `False`. 
* How many linear terms are in each of the new feature set?
* How many squared terms are in each of the new feature set?
* How many interaction terms are in each of the new feature set? Give an example of one of the interaction terms. 

b) Now generate 3 more models / design matrices. Again it should include all quadratic terms and 2-way interactions - but each model should drop one of the features
* Second Model: Drop `standing_tackle`
* Third Model: Drop `composure`
* Fourth model: Drop `marking`

Hint: For these models, create the design matrix without the aforementioned features and then apply polynomial expansion to the remaing features. 


In [ ]:
# a) Make the new expanded design matrix 
poly = sk.preprocessing.PolynomialFeatures(2,include_bias=False)


**Written answer here**

 47/48 linear linear terms

 48x47/2  (49x48/2) interaction terms including includes the squared terms. *Computed using trianglular number*

 Total of 1128 (1176) terms

In [ ]:
# Make design matrices without one of the features
Xwithout_standing = Xtrain.drop(['standing_tackle'], axis='columns')
Xwithout_standing = poly.fit_transform(Xwithout_standing)
Xwithout_composure = Xtrain.drop(['composure'], axis='columns')
Xwithout_composure = poly.fit_transform(Xwithout_composure)
Xwithout_marking= Xtrain.drop(['marking'], axis='columns')
Xwithout_marking = poly.fit_transform(Xwithout_marking)
X_new_train = poly.fit_transform(Xtrain)

## Question 4: Evaluating the backward feature search (15 pts)
In this question, you have to use the pipeline created in question 2 and apply it to each of the models in question 3. Use 10-fold cross validation to report the validation error on the models using mean squared error as the metric. <br>
Show all the steps of the process and compare and analyze the results using the validation error reported. For this first step in the backwards search, which feature would you drop? 

In [ ]:
# Your code here
cv_score=cross_val_score(model_pipeline, X_new_train, ytrain, cv=10, scoring='neg_mean_squared_error')
print('base model', -cv_score.mean())

cv_score=cross_val_score(model_pipeline, Xwithout_standing, ytrain, cv=10, scoring='neg_mean_squared_error')
print('model without standing tackle', -cv_score.mean())


cv_score=cross_val_score(model_pipeline, Xwithout_composure, ytrain, cv=10, scoring='neg_mean_squared_error')
print('model without composure', -cv_score.mean())

cv_score=cross_val_score(model_pipeline, Xwithout_marking, ytrain, cv=10, scoring='neg_mean_squared_error')
print('model without marking', -cv_score.mean())

**Written answer here** 
I would drop the base model that includes all the features as it is the worst performing model

## Question 5: Applying Ridge Regression (10 pts)
Build a pipeline that performs scaling and fits the ridge regression on the data that includes the polynomial expansion of all the features. The ridge parameter ($\lambda$ or `alpha` in sklearn) should be set to 0.5. Use the pipeline to report the validation error using mean square error metric. Use 10-fold cross validation. 

In [ ]:
# Your code here
pip = Pipeline([
    ('standardize', StandardScaler()),
    ('reg', Ridge(alpha=0.5, fit_intercept=True))
])
cv_scores = cross_val_score(pip,
                           X_new_train,
                           ytrain,
                           cv=10,
                           scoring='neg_mean_squared_error')
meancv=-cv_scores.mean()
print(meancv)

## Question 6: Tune the Ridge coefficient for the 2nd-order model (15 pts)
Perform the search going from $\lambda = \exp(-8), \cdots, \exp(6)$ in 15 evenly spaced increments on the log scale. 

For each setting of lambda, calculate the training error when fitting the regularized model to the entire trainign data set, and the prediction error by studying the performance on the left-out part using 10-fold cross-validation. (*Note this calculation can take a bit, be patient*)

Plot the mean squared training error and mean squared validation error as a function of $\log(\lambda)$. 

Note: Although you can ultimately use `GridSearchCV` from sklearn, in this task you need to program a for-loop interating over all the levels of $\lambda$.  

### Questions: 

What is the best regularization parameter? 

Why does the validation error increase as $\lambda \rightarrow 0$, while the training error decreases from the optimal value?  Why does both the training and the validation error increase when $\lambda \rightarrow \infty$?  Answer in terms of the bias variance trade off and model complexity.


In [ ]:
# Your code here
params = {'reg__alpha': np.exp(np.linspace(-8,6,15))}
gscv = GridSearchCV(pip, param_grid=params, cv=10, scoring = 'neg_mean_squared_error', refit=True)
gscv.fit(X_new_train, ytrain)

results = pd.DataFrame(gscv.cv_results_)

plt.scatter( np.linspace(-8, 6,15), -results.mean_test_score)
plt.xlabel(r'$\log(\lambda)$')

**Written answer here**

The best reg parameter can be found using `gscv.best_params_`. 

As $\lambda \rightarrow \infty$, the model becomes increasingly biased increasingt the RMSE.  RMSE does not increase as $\lambda \rightarrow 0$ because it has a low variance high bias model which is a special case of linear regression.

## Question 7: Test error  (10 pts)
Now fit the model using ridge regression, using the lambda-value that you determined works best (in terms of crossvalidated mse) from Question 6. Fit the model on the whole training set. 

Report the mean squared error on the test data - along with the 95% confidence interval, determined with the central limit theorem (remember assignment 5). 

In [ ]:
# Your code here
X_new_test = poly.fit_transform(Xtest)
ypred = gscv.predict(X_new_test)
sqerr = (ytest-ypred)**2 
test_error = sqerr.mean() 
test_ci = test_error + 1.96 * np.std(sqerr) / np.sqrt(len(sqerr)) * np.array([-1, 1])

print("Mean test error: ",test_error)
print("95% CI: ",test_ci)

ss = (ytest-ytest.mean())**2 
print("Proportion variance predicted: ",1-test_error/ss.mean())
print("95% CI: ",[1-test_ci[1]/ss.mean(), 1-test_ci[0]/ss.mean()])

## Question 8: Lasso Regression (10 pts)
That's great! You can achieve a really good prediction accuracy with much less data than in Week 5. Impressive! 

Now the problem is that the model is really hard to interpret and explain to clients- the importance of each feature is not easily apparent. So let's build a simpler model, which is only based on the first nine features:
* age
* height_cm
* weight_kg 
* pac: ???  
* sho: shooting 
* pas: passing 
* dri: dribble 
* def: defense
* phy: Physiological VO2 max

Build a design matrix using only these nine features. Standardize the design matrix using the standard scalar. 
Then use `sklearn.linear_model.lasso_path` to create a plot of the regression coefficients against the log-regularization parameter (see `Lab06_followalong`). Note that it is standard practice to plot on the x-axis the negative log-lambda values, such that the high regularization (and hence the simpler models) are on the left. 

Which of the 9 variables drops out of the predicitive model first? Which one is retained for the longest time? 

In [ ]:
# Make a new, reduced design matrix 
# Aformentioned columns
x = df[['age','height_cm','weight_kg','pac','sho','pas','dri','def','phy']]

#Standardize, unsure how to standardize X properly as when it is fit the coef are 0, 
# and would take an unreasonable amount of time to run ?

poly = sk.preprocessing.PolynomialFeatures(6)
X = poly.fit_transform(x)
scaler = sk.preprocessing.StandardScaler(with_mean=True, with_std=True)
X = scaler.fit_transform(X)

# Create a lasso path
eps = 5e-3 # The smaller eps, the longer the path  
lambda_lasso, coefs_lasso, _ = sk.linear_model.lasso_path(X, y, eps, n_alphas=100, alphas= None, fit_intercept=False)

print(f"minmum regularization parameter : {np.amin(lambda_lasso)}")
print(f"maximum regularization parameter : {np.amax(lambda_lasso)}")


colors = ['b', 'r', 'g', 'c', 'k','y','purple','orange','pink']
neg_log_lambda = -np.log(lambda_lasso)

for i in range(9):
    l1 = plt.plot(neg_log_lambda, coefs_lasso[i,], c=colors[i])
    label = f'Poly:{i}'
    plt.annotate(label, # this is the text
                 (neg_log_lambda[-1],coefs_lasso[i,-1]), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center',
                 color = colors[i]) # horizontal alignment can be left, right or center



**Written answer here**


## Question 9: Tuning the lasso coefficient 
We now want to find a model that is both simple (explainable), but at the same time yields still relatively decent predictions. To assess this, vary the regularization constant of the lasso model between exp(2) end exp(-3.5). That is, vary negative log lambda between -2 and 3.5 in even steps. 
Plot the crossvalidation error (mean squared error) against negative-log-lambda of the model. 

Hint: You can either program a for-loop as in Question 6, or use the function `GridSearchCV`. 

What is the simplest model that still gives you a expected validation error of below 15? 
If you wanted the best validation error, what $\lambda$ would you need to use?

In [ ]:
import math
# Your code here
# Pipeline
# pipe2 = Pipeline([('poly', PolynomialFeatures()),
                 # ('fit', linear_model.Lasso())])
# Params
# params = {'reg__alpha': np.exp(np.linspace(math.exp(2),math.exp(-3.5),15))}

#'Lasso': GridSearchCV(linear_model.Lasso(), 
                               # param_grid=lasso_params).fit(df[X], df[Y]).best_estimator_,
# for loop that applies exp(2) on even exp(-3.5) on odd

# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
# grid = dict()
# grid['alpha'] = arange(0, 1, 0.01)
# define search
# search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
# results = search.fit(X, y)




**Written answer here**

## Question  10: Lasso vs. Ridge
In this quesiton, we will compare Ridge regression and Lasso solutions on the model defined in Question 8. Make sure you are using a standardized design matrix for this task. 

Fit the model using Ridge (L2- regularization, $\lambda = exp(-3)$) and Lasso (L1-regularization, $\lambda = exp(-0.5)$. Then print out the regression coefficients for each of the nine features in the design matrix. 

Based on the coefficients, which one is the most important feature in the Ridge vs. Lasso solution? How many features are contributing to the prediction for each solution? How can this difference be explained? 


In [ ]:
# Your code here
# Know how to create it but can't run without standardized X

#Ridge 
ridge = sk.linear_model.Ridge(alpha=math.exp(-3),fit_intercept=True)
ridge.fit(X,y)

#Lasso
las = sk.linear_model.Lasso(alpha=math.exp(-0.5),fit_intercept=True)
las.fit(X,y)

(ridge.coef_,las.coef_)


**Written answer here**